In [10]:
import torch.nn.functional as F  # functions of the neural network library
import load_dataset as load  # module with function to load data
from Conv_Layer import *
from Pooling import *
import gc
from Dense import *
from sklearn.decomposition import PCA

In [53]:
I = 12
O = I//2
n = 2*I
k = 2
K = I//4

device = torch.device("mps")
batch_size = 144  # the number of examples per batch
train_loader, test_loader, dim_in, dim_out = load.load_MNIST(batch_size=batch_size)
scala = 10
init_reduced_loader = reduce_MNIST_dataset(train_loader, scala)
reduced_loader = filter_dataloader(init_reduced_loader, classes=[0, 1])
list_gates = [(i, j) for i in range(O-1) for j in range(i+1, O-1)]

full_model = nn.Sequential(Conv_RBS_density_I2(I,K,device), Pooling_2D_density(I,O,device),
                           Conv_RBS_density_I2(I//2,K,device), Pooling_2D_density(I//2,O//2,device),
                           Basis_Change_I_to_HW_density(O//2, device), Dense_RBS_density(O//2, list_gates, device))

loss_function = torch.nn.MSELoss(reduction='sum')
optimizer = torch.optim.Adagrad(full_model.parameters(), lr=1e-1, lr_decay=1e-6, weight_decay=0, initial_accumulator_value=1e-6, eps=1e-10)

def train_net(network, train_loader, loss_function, optimizer, device):
    network.train()
    train_loss = 0
    train_accuracy = 0
    train_out = []
    for batch_idx, (data, target) in enumerate(train_loader):
        if data.size()[0] < batch_size:
            break
        # batch_data tensor size: batch_size * 784
        # init_density_matrix tensor size: batch_size * 784 * 784
        
        images_flattened = data.view(data.shape[0], -1).numpy() # batch * 784
        pca = PCA(n_components=144)
        data = torch.from_numpy(pca.fit_transform(images_flattened)) # batch * n_components

        init_density_matrix = to_density_matrix(F.normalize(data, p=2, dim=1).to(device), device)
        out_network = network(init_density_matrix) # out tensor size: batch * 15 * 15

        # training
        targets = get_batch_projectors(target, batch_size, int(binom(O,2)), 10, device)
        loss = loss_function(out_network,targets)
        train_loss += loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # predict digital number
        predict_number_vector = get_predict_number_vector(out_network, device)
        predict_number = torch.argmax(predict_number_vector, dim=1).to(device)
        acc = predict_number.eq(target.to(device).view_as(predict_number).to(device)).sum().item()
        train_accuracy += acc
        
        train_out.append(out_network)

        # delete variable to free memory
        del out_network
        gc.collect()

    train_accuracy /= len(train_loader.dataset)
    train_loss /= (batch_idx + 1)
    return train_accuracy, train_loss, train_out
    

In [55]:
for epoch in range(20):
    train_accuracy, train_loss,  train_out = train_net(full_model, reduced_loader, loss_function, optimizer, device)
    print(f'Epoch {epoch}: Loss  = {train_loss:.6f}, accuracy = {train_accuracy*100:.4f} %')

66
59
63
70
71
62
63
55
Epoch 0: Loss  = 93.693823, accuracy = 40.3009 %
55
63
54
90
78
61
67
88
Epoch 1: Loss  = 83.972526, accuracy = 44.0222 %
83
77
70
76
78
80
85
72
Epoch 2: Loss  = 79.877822, accuracy = 49.1686 %
80
78
90
86
69
72
71
80
Epoch 3: Loss  = 76.810446, accuracy = 49.5645 %
82
80
77
81
76
83
84
76
Epoch 4: Loss  = 74.276599, accuracy = 50.5938 %


KeyboardInterrupt: 

In [48]:
import torch
from torch.utils.data import DataLoader, TensorDataset


def filter_dataloader(dataloader, classes=[0, 1]):
    filtered_data = []
    filtered_targets = []

    for data, target in dataloader:
        mask = target == classes[0]
        for c in classes[1:]:
            mask = mask | (target == c)

        filtered_data.append(data[mask])
        filtered_targets.append(target[mask])

    # Concatenate all collected data and targets
    filtered_data = torch.cat(filtered_data, dim=0)
    filtered_targets = torch.cat(filtered_targets, dim=0)

    # Create a new TensorDataset and DataLoader from the filtered data and targets
    filtered_dataset = TensorDataset(filtered_data, filtered_targets)
    filtered_dataloader = DataLoader(filtered_dataset, batch_size=dataloader.batch_size, shuffle=False)

    return filtered_dataloader

# Example usage:
# Assuming `original_dataloader` is your MNIST DataLoader
filtered_dataloader = filter_dataloader(reduced_loader, classes=[0, 1])
# for batch_idx, (data, target) in enumerate(filtered_dataloader):
#     print()

for batch_idx, (data, target) in enumerate(init_reduced_loader):


tensor(0)
tensor([[[-0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242,
          -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242,
          -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242,
          -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242],
         [-0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242,
          -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242,
          -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242,
          -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242],
         [-0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242,
          -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242,
          -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242,
          -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242],
         [-0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242,
          -0.4242, -0.4242, -0.4242, -